
# NVIDIA NIMs with Tool Calling for Agents

This notebook will use a [NVIDIA Llama 3.1 NIM](https://developer.nvidia.com/blog/supercharging-llama-3-1-across-nvidia-platforms/) with tool-calling agent capabilities in generative AI solutions. As mentioned in this [Introductory Blog on LLM Agents](https://developer.nvidia.com/blog/introduction-to-llm-agents/), agents can be described as AI systems that use LLMs to reason through a problem, create a plan to solve the problem, execute the plan with the help of a set of tools, and use memory to store meaningful context of the system state. 

The notebook is designed to provide an intro to merely one of the capabilities of agent systems: **tool calling**. 

**Tools** are interfaces that accept input, execute an action, and then return a result of that action in a structured output according to a pre-defined schema. They often encompass external API calls that the agent can use to perform tasks that go beyond the capabilities of the LLM, but do not have to be external API calls. For example, to get the current weather in San Diego, a weather tool might be used. Or to get the current score of the 49ers game, a generic web search tool or ESPN tool might be used. 

## What is NVIDIA NIM and How do They Support Tool Calling for Agents?
### What is NIM?
NIM supports models across domains like chat, embedding, and re-ranking models 
from the community as well as NVIDIA. These models are optimized by NVIDIA to deliver the best performance on NVIDIA 
accelerated infrastructure and deployed as a NIM, an easy-to-use, prebuilt containers that deploy anywhere using a single 
command on NVIDIA accelerated infrastructure. If you're new to NIMs with LangChain, check out the [documentation](https://python.langchain.com/v0.2/docs/integrations/providers/nvidia/).

Now, NIMs support tool calling, also known as "function calling" for models that have the aforementioned capability. 

This notebook will demonstrate a model that supports function calling, [Llama 3.1 8b-instruct](https://build.nvidia.com/meta/llama-3_1-8b-instruct). 

### What does it mean for NIM to support tool usage?
In order to support tool usage in an agent workflow, first an LLM must be trained to detect when a function should be called and output a structured response like JSON that contains the function to be called and its arguments. 

Next, the model is packaged as a NIM, meaning it's optimized to deliver best performance on NVIDIA accelerated infrastructure and easy to deploy as well as use. This microservice packaging also uses OpenAI compatible APIs, so developers can build world-class generative AI agents with ease.

Let's see how to use tools in a couple of examples.

### Prerequisites
TODO: clarify which one is NGC key and which one is api key for hosted access and provide linked access when possible
Before you start this notebook, ensure that you either have:

- A) an NGC key available that is able to access the Llama3.1 NIM on NGC or 
- B) a build.nvidia.com key to access the Llama3.1 NIM hosted by NIVIDA.

If you are planning on doing A, hosting a Llama3.1 NIM on a Brev instance, please consult the instructions below. 

#### Install Docker and NGC CLI

In [ ]:
! unzip -q -o ngccli_linux.zip

In [45]:
%%bash
# todo: change to wget? 
chmod +x ./assets/setup-ngc-barebones.sh
./assets/setup-ngc-barebones.sh

Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.
Installing NGC CLI...
Installing NGC CLI...


groupadd: group 'docker' already exists


#### Download Llama 3.1 8B NIM
Let's download the NIM from NGC. 

Note this cell might take a few minutes as it pulls the NIM. 

In [48]:
%%bash

chmod +x ./assets/setup-llama3.1-nim
export NGC_API_KEY="nvapi-_XXX"
./setup-nim

chmod: cannot access './assets/setup-llama3.1-nim': No such file or directory
bash: line 4: ./setup-nim: No such file or directory


CalledProcessError: Command 'b'\nchmod +x ./assets/setup-llama3.1-nim\nexport NGC_API_KEY="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"\n./setup-nim\n'' returned non-zero exit status 127.

##  🔨 Tool Usage -- Web Search

Since a LLM does not have access to the most up-to-date information on the Internet, [Tavily Search](https://docs.tavily.com/docs/tavily-api/introduction) acts as a tool to provide a generative AI application with real-time online information.  Tavily is a search engmine that is optimized for AI developers and AI agents. A singular API call abstracts searching, scraping, filtering, and extracting relevant information from online sources. 

We'll enhance our NIM, [Llama 3.1-8b-instruct](https://build.nvidia.com/meta/llama-3_1-8b-instruct), with Tavily search. 

Install pre-requesites. 

In [ ]:
%pip install -U langchain langgraph langchain-nvidia-ai-endpoints langchain-community langchain-openai tavily-python geocoder

If you're using NVIDIA hosted NIMs, you'll need to use an API key which you can setup below. Follow [NVIDIA NIMs LangChain documentation](https://python.langchain.com/v0.2/docs/integrations/chat/nvidia_ai_endpoints/) for more information on accessing and using NIMs. 

In [ ]:
import getpass
import os

os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter your NVIDIA API key: ")

Declare your model that supports tool calling. In this example, we use [Llama 3.1-8b-instruct](https://build.nvidia.com/meta/llama-3_1-8b-instruct). 

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

Initialize [Tavily Tool](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/)

Note that this requires an API key - they have a free tier, but if you don't have one or don't want to create one, you can always ignore this step or use a different tool. 

Once you create your API key, you will need to set it in the environment.

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Declare a single tool, Tavily search
tools = [TavilySearchResults(max_results=1)]

Create [ReAct agent](https://python.langchain.com/v0.2/docs/concepts/#react-agents), prebuilt in [LangGraph](https://langchain-ai.github.io/langgraph/#overview). 

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain.callbacks.tracers import ConsoleCallbackHandler

app = create_react_agent(llm, tools)

Run agent; a callback is passed to provide more verbose output.

In [ ]:
query = "What is LangChain?"
messages = app.invoke({"messages": [("human", query)]}, config={'callbacks': [ConsoleCallbackHandler()]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

## 🔨 Tool Usage -- Adding on a Custom Tool

Let's see how to [define a custom tool](https://python.langchain.com/v0.2/docs/how_to/custom_tools/) for your NIM agent and how it handles multiple tools.  

We'll enhance the NIM with Tavily search with some custom tools to determine a user's current location (based on IP address) and return a latitude and longitude. We will use these tools to have Tavily look up the weather in the user's current location.

First, let's create a custom tool to determine a user's location based off IP address. 

In [ ]:
import geocoder
from langchain.tools import tool
from typing import Tuple

@tool
def get_current_location() -> list:
    """Return the current location of the user based on IP address"""
    loc = geocoder.ip('me')
    return loc.latlng    

Let's update the tools to use the Tavily tool delcared earlier and also add the `get_current_location` tool.

In [ ]:
# Declare two tools: Tavily and custom get_current_location tool.
tools = [TavilySearchResults(max_results=1), get_current_location]

We already declared our LLM, so we don't need to redeclare it. However, we do want to update the agent to have the updated tools.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain.globals import set_verbose
from langchain.callbacks.tracers import ConsoleCallbackHandler

set_verbose(True) # verbose output to follow function calling

query = "What is the current weather where I am?"
app = create_react_agent(llm, tools)


messages = app.invoke({"messages": [("human", query)]}, config={'callbacks': [ConsoleCallbackHandler()]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

In order to execute this query, first a tool to get the current location needs to be called. Then a tool to get the current weather at that location needs to be called. 
Finally, the result is returned to the user.

Below, you can see a diagram of the application's graph. The agent continues to use tools until the query is resolved.

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

## Conclusion
You've now seen how to use NIMs to do tool calling, an important capability of agents. As mentioned earlier, tools are just one part of agent capabilities, so check out other notebook so see how tools can be used with othe techniques to create agent workflows.

If you're ready to explore more complicated agent workflows, check out [this blog](https://developer.nvidia.com/blog/build-an-agentic-rag-pipeline-with-llama-3-1-and-nvidia-nemo-retriever-nims/) on how to improve your RAG pipeline with agents with Llama 3.1 and NVIDIA NemMo Retriever NIMs.